In [159]:
from sympy import *
x,y = symbols('x y')
c = symbols('C', constant=True)

f = Function('f')(x)

# two linear operators
#
# A1 corresponds to   Af = xf'
# A2 corresponds to   Af = (x+1)f'
# Ac corresponds to   Af = (x+C)f'
def A1(f):
    return x * f.diff(x,1)

def A2(f):
    return (x+1) * f.diff(x,1)

def Ac(f):
    return (x+c) * f.diff(x,1)

# C computes the iterated function composition
#
# ( A o A o ... o A o f)
#   \............../
#    n times
#
def C(A,f,n):
    res = f
    for i in range(n):
        res = expand(A(res))
    return res

# convert f(x) (d/dx)^k -> y^k
#
# this function just searches for all derivatives f^(k)(x) and replaces them with y**k.
# the resulting expression is easier to read and handle.
def T(A):
    res = A
    if isinstance(A,Add):
        for u in A.args:
            u1 = u.args
            for i1 in range(len(u1)):
                if isinstance(u1[i1],Derivative):
                    u2 = u1[i1].args
                    if u2[0]==f:
                        k = u2[1][1]
                        res = res.replace(f.diff(x,k),y**k)
    elif isinstance(A,Mul):
        # in this branch we only have one term
        # the +1-1 helps to reuse the Add branch
        res = T(A+1)-1
    return collect(res,y)

N=13

# http://oeis.org/search?q=2%2C5%2C15%2C52%2C203%2C877%2C4140%2C21147%2C115975&sort=&language=english&go=Search
print("\nA1 case")
U0=["Af=xf'"]
for n in range(N):
    An = C(A1,f,n)
    TAn = T(An)
    s = TAn.subs({x:1,y:1})
    U0.append(s)
    print(s)

# http://oeis.org/search?q=6+22+94+454+2430+14214+89918&sort=&language=english&go=Search
print("\nA2 case")
U1=["Af=(x+1)f'"]
for n in range(N):
    An = C(A2,f,n)
    TAn = T(An)
    s = TAn.subs({x:1,y:1})
    U1.append(s)
    print(s)

print("\nAc case")
U2=["Af=(x+1/2)f'"]
for n in range(N):
    An = C(Ac,f,n)
    TAn = T(An)
    s = floor(TAn.subs({x:1,y:1,c: S(1)/2}).evalf())
    U2.append(s)
    print(s)


A1 case
f(1)
1
2
5
15
52
203
877
4140
21147
115975
678570
4213597

A2 case
f(1)
2
6
22
94
454
2430
14214
89918
610182
4412798
33827974
273646526

Ac case
floor(f(1))
1
3
11
42
177
829
4250
23666
141898
909709
6199837
44700218


In [160]:
# https://en.wikipedia.org/wiki/Bell_number#Growth_rate
def bbell(n):
    return ceiling(((0.792*n / log(n+1))**n).evalf())

U3 = ["B_n bound"]
U4 = ["B_n between"]
U = [U0,U1,U2,U3,U4]
for n in range(2,N):
    U[3].append(bbell(n))
    U[4].append(bbell(n) >= U[2][n] and bbell(n) <= U[1][n])

from tabulate import tabulate
print(tabulate(map(list, zip(*U))))


------  ----------  ------------  ---------  -----------
Af=xf'  Af=(x+1)f'  Af=(x+1/2)f'  B_n bound  B_n between
f(1)    f(1)        floor(f(1))   3          False
1       2           1             6          True
2       6           3             16         True
5       22          11            53         True
15      94          42            213        True
52      454         177           958        True
203     2430        829           4782       True
877     14214       4250          26108      True
4140    89918       23666         154508     True
21147   610182      141898        983753     True
115975  4412798     909709        6697792    True
------  ----------  ------------  ---------  -----------
